In [1]:
import pandas as pd
import yfinance as yf

In [43]:
df = pd.read_csv('matched_reconciled.csv', index_col='idx')

In [44]:
df.head()

,name,count,matched_f,Ticker_f,source_f,comment,yf_ticker
idx,,,,,,,
0,glaxosmithkline,1706,glaxosmithkline,GSK,Stoxx,y,GSK.L
1,merck sharp & dohme corp.,1360,merck co,MRK,Russell,y,MRK
2,pfizer,1243,pfizer,PFE,Russell,y,PFE
3,eli lilly and company,1031,eli lilly,LLY,Russell,y,LLY
4,novartis pharmaceuticals,886,novartis,NOVN,Stoxx,y,NOVN.SW


### Let's test the YF API for splits and dividend adjustments

#### 1. AAPL did a 7-for-1 split on 6/9/2014, let's see if this was adjusted properly.

In [11]:
data = yf.download("AAPL", start="2014-06-01", end="2014-06-30")

[*********************100%***********************]  1 of 1 completed


In [12]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-06-02,90.565712,90.690002,88.928574,89.807144,81.559891,92337700
2014-06-03,89.779999,91.248573,89.750000,91.077141,82.713257,73177300
2014-06-04,91.062859,92.555717,90.872856,92.117142,83.657745,83870500
2014-06-05,92.314285,92.767143,91.801430,92.478569,83.985992,75951400
2014-06-06,92.842857,93.037140,92.067146,92.224289,83.755051,87484600
2014-06-09,92.699997,93.879997,91.750000,93.699997,85.095238,75415000
2014-06-10,94.730003,95.050003,93.570000,94.250000,85.594727,62777000
2014-06-11,94.129997,94.760002,93.470001,93.860001,85.240562,45681000
2014-06-12,94.040001,94.120003,91.900002,92.290001,83.814751,54749000


The split looks like it does not cause a large jump in the stock price

### Let's see if they handle the dividend properly

In [14]:
aapl = yf.Ticker('AAPL')

In [22]:
aapl.dividends[-2:]

Date
2019-11-07    0.77
2020-02-07    0.77
Name: Dividends, dtype: float64

In [23]:
data = yf.download("AAPL", start="2019-11-01", end="2020-02-12")

[*********************100%***********************]  1 of 1 completed


#### let's see the adjustment on 2/6 before the 2/7 div:

In [27]:
data[-5:]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-05,323.519989,324.760010,318.950012,321.450012,320.688934,29706700
2020-02-06,322.570007,325.220001,320.260010,325.209991,324.440002,26356400
2020-02-07,322.369995,323.399994,318.000000,320.029999,320.029999,29421000
2020-02-10,314.179993,321.549988,313.850006,321.549988,321.549988,27337200
2020-02-11,323.600006,323.899994,318.709991,319.609985,319.609985,23580800


In [34]:
320.029999 / 324.440002

0.9864073388829531

In [35]:
320.029999 / (325.209991 - 0.77)

0.9864073723266746

### Let's see the adjustment on 11/6 before the 11/7 div

In [31]:
data[:5]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-11-01,249.539993,255.929993,249.160004,255.820007,254.450378,37781300
2019-11-04,257.329987,257.850006,255.380005,257.500000,256.121368,25818000
2019-11-05,257.049988,258.190002,256.320007,257.130005,255.753372,19974400
2019-11-06,256.769989,257.489990,255.369995,257.239990,255.862762,18966100
2019-11-07,258.739990,260.350006,258.109985,259.429993,258.815735,23735100


In [36]:
258.815735 / 255.862762

1.0115412378765771

In [38]:
259.429993 / (257.239990 - 0.77)

1.0115413230218475

### Excellent, the Adj Close accounts for DIVs and Splits properly

# How to load foreign stock data?

In [47]:
df['source_f'].unique()

array(['Stoxx', 'Russell', 'Japan', 'Australia'], dtype=object)

In [48]:
df[df['source_f']=='Australia']

,name,count,matched_f,Ticker_f,source_f,comment,yf_ticker
idx,,,,,,,
79,csl behring,44,csl,CSL,Australia,manual,CSL.AX
333,cochlear,9,cochlear,COH,Australia,y,COH.AX


## Ready to load all stock data for 10 years, that is where the posted results are concentrated

In [52]:
sd = "2009-12-31"
ed = "2020-03-20"
ticker_string = " ".join(df['yf_ticker'].unique())

data = yf.download(ticker_string, start=sd, end=ed)

[*********************100%***********************]  142 of 142 completed


In [55]:
data['Adj Close'].fillna(method='ffill', limit=10)

,4188.T,4502.T,4503.T,4506.T,4507.T,4519.T,4523.T,4568.T,4578.T,4768.T,...,TXMD,UCB.BR,UTHR,VIFN.SW,VNDA,VRTX,XENT,XERS,ZBH,ZGNX
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130.00,22.317200,52.650002,9.026128,11.25,42.849998,NaN,NaN,55.095654,NaN
2010-01-04,296.624176,2609.193359,292.526825,847.636475,1637.445312,1452.224365,2491.107178,1453.463257,NaN,330.747467,...,150.00,22.405031,53.439999,9.236738,11.67,44.240002,NaN,NaN,55.943844,NaN
2010-01-05,300.398010,2622.748291,295.055969,851.965393,1640.715210,1431.383057,2483.854492,1461.666748,NaN,335.747711,...,150.00,22.599792,53.270000,9.302929,11.28,42.779999,NaN,NaN,57.714798,NaN
2010-01-06,300.398010,2622.748291,295.055969,858.026184,1640.715210,1443.888184,2509.236572,1461.666748,NaN,342.176941,...,137.00,22.634163,54.099998,9.224703,10.99,42.029999,NaN,NaN,57.696159,NaN
2010-01-07,303.417145,2663.410645,295.055969,850.233826,1599.840454,1431.383057,2494.732666,1476.581909,NaN,342.891418,...,136.00,22.832739,55.470001,9.387174,10.94,41.500000,NaN,NaN,59.019741,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-16,606.799988,3033.000000,1470.500000,1245.000000,4640.000000,10500.000000,6706.000000,5528.000000,3338.0,3770.000000,...,1.26,69.040001,79.430000,114.900002,8.10,199.770004,9.50,1.57,92.000000,19.510000
2020-03-17,605.400024,3030.000000,1480.000000,1222.000000,4675.000000,11055.000000,6615.000000,5742.000000,3343.0,3955.000000,...,1.27,65.839996,80.559998,102.050003,8.00,210.440002,9.45,1.79,86.160004,20.049999
2020-03-18,615.299988,2900.000000,1474.000000,1223.000000,4401.000000,10710.000000,6428.000000,5910.000000,3240.0,3965.000000,...,1.00,63.419998,79.389999,102.900002,7.50,212.979996,8.95,1.49,80.629997,17.799999
